In [2]:
# ================================
# SCRIPT TO CALCULATE AND SAVE SALES AND QUANTITY SUMS
# FOR RECENT AND PAST TIME FRAMES TO DATABASE TABLES
# ================================
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
import numpy as np
from datetime import datetime

# ================================
# CONFIGURATION
# ================================
CONFIG = {
    # --- MSSQL PULL ---
    'sql_query': """
        SELECT
            TranDate_DDMMYYYY AS order_date,
            NetSales AS sales_value,
            qty AS quantity
        FROM TEST1.dbo._Gift_from_Carpenters
        WHERE NetSales IS NOT NULL
    """,
    'server': '100.99.225.51',
    'database': 'TEST1',
    'username': 'SA',
    'password': 'a31536000',
    'driver': '{ODBC Driver 17 for SQL Server}',
    'encrypt': 'yes',
    'trust_server_certificate': 'yes',
    # --- TIMEOUTS (Critical for bulk ops) ---
    'connection_timeout': 120, # 2 minutes
    'query_timeout': 300, # 5 minutes
    # --- DATA SETTINGS ---
    'date_column': None,
    'value_column': None,
    'quantity_column': None,
    'date_format': '%d/%m/%Y',
    # --- SUMMARY TABLES ---
    'past_summary_table': 'past_timeframe_performance_summary',
    'recent_summary_table': 'recent_timeframe_performance_summary'
}

# ================================
# 1. LOAD FROM MSSQL
# ================================
def load_data(config):
    conn_str = (
        f"DRIVER={config['driver']};"
        f"SERVER={config['server']};"
        f"DATABASE={config['database']};"
        f"UID={config['username']};"
        f"PWD={config['password']};"
        f"Encrypt={config['encrypt']};"
        f"TrustServerCertificate={config['trust_server_certificate']};"
    )
    try:
        print("Connecting to MSSQL...")
        conn = pyodbc.connect(conn_str)
        df = pd.read_sql(config['sql_query'], conn)
        conn.close()
        print(f"Loaded {len(df):,} rows from DB")
        # Auto-detect columns
        date_col = config['date_column'] or 'order_date'
        value_col = config['value_column'] or 'sales_value'
        quantity_col = config['quantity_column'] or 'quantity'
        df[date_col] = pd.to_datetime(df[date_col], format=config['date_format'], errors='coerce')
        df[value_col] = pd.to_numeric(df[value_col], errors='coerce')
        df[quantity_col] = pd.to_numeric(df[quantity_col], errors='coerce')
        df = df.dropna(subset=[date_col, value_col, quantity_col])
        print(f"Cleaned data: {len(df):,} rows")
        return df, date_col, value_col, quantity_col
    except Exception as e:
        print(f"DB Load Failed: {e}")
        return None, None, None, None

# ================================
# 2. CALCULATE PERIOD SUMS
# ================================
def calculate_period_sums(df, date_col, value_col, quantity_col, current_date, is_past=False):
    shift = pd.DateOffset(years=1) if is_past else pd.DateOffset(years=0)
    sums = {}

    # Previous day
    prev_day = current_date - pd.Timedelta(days=1) - shift
    mask = (df[date_col] == prev_day)
    sums['previous_day'] = {
        'total_sales': df[mask][value_col].sum(),
        'total_quantity': df[mask][quantity_col].sum()
    }

    # Previous week (last 7 days up to yesterday)
    prev_week_start = current_date - pd.Timedelta(days=7) - shift
    prev_week_end = current_date - pd.Timedelta(days=1) - shift
    mask = (df[date_col] >= prev_week_start) & (df[date_col] <= prev_week_end)
    sums['previous_week'] = {
        'total_sales': df[mask][value_col].sum(),
        'total_quantity': df[mask][quantity_col].sum()
    }

    # Previous month (last full calendar month)
    prev_month_end = (current_date - pd.Timedelta(days=1) - shift).replace(day=1) - pd.Timedelta(days=1)
    prev_month_start = prev_month_end.replace(day=1)
    mask = (df[date_col] >= prev_month_start) & (df[date_col] <= prev_month_end)
    sums['previous_month'] = {
        'total_sales': df[mask][value_col].sum(),
        'total_quantity': df[mask][quantity_col].sum()
    }

    # Previous quarterly (last full quarter)
    current_month = (current_date - shift).month
    quarter = (current_month - 1) // 3 + 1
    prev_quarter = quarter - 1 if quarter > 1 else 4
    prev_quarter_year = (current_date - shift).year if quarter > 1 else (current_date - shift).year - 1
    if prev_quarter == 1:
        prev_quarter_start = pd.to_datetime(f"{prev_quarter_year}-01-01")
        prev_quarter_end = pd.to_datetime(f"{prev_quarter_year}-03-31")
    elif prev_quarter == 2:
        prev_quarter_start = pd.to_datetime(f"{prev_quarter_year}-04-01")
        prev_quarter_end = pd.to_datetime(f"{prev_quarter_year}-06-30")
    elif prev_quarter == 3:
        prev_quarter_start = pd.to_datetime(f"{prev_quarter_year}-07-01")
        prev_quarter_end = pd.to_datetime(f"{prev_quarter_year}-09-30")
    else:
        prev_quarter_start = pd.to_datetime(f"{prev_quarter_year}-10-01")
        prev_quarter_end = pd.to_datetime(f"{prev_quarter_year}-12-31")
    mask = (df[date_col] >= prev_quarter_start) & (df[date_col] <= prev_quarter_end)
    sums['previous_quarter'] = {
        'total_sales': df[mask][value_col].sum(),
        'total_quantity': df[mask][quantity_col].sum()
    }

    # Past year (last 365 days up to yesterday)
    prev_year_start = current_date - pd.Timedelta(days=365) - shift
    prev_year_end = current_date - pd.Timedelta(days=1) - shift
    mask = (df[date_col] >= prev_year_start) & (df[date_col] <= prev_year_end)
    sums['past_year'] = {
        'total_sales': df[mask][value_col].sum(),
        'total_quantity': df[mask][quantity_col].sum()
    }

    # To DataFrame
    summary_df = pd.DataFrame({
        'period': list(sums.keys()),
        'total_sales': [v['total_sales'] for v in sums.values()],
        'total_quantity': [v['total_quantity'] for v in sums.values()],
        'calculated_date': current_date.date()
    })
    return summary_df

# ================================
# 3. PUSH SUMMARY TO DB
# ================================
def push_summary_to_db(summary_df, config, table_name):
    conn_str = (
        f"DRIVER={config['driver']};"
        f"SERVER={config['server']};"
        f"DATABASE={config['database']};"
        f"UID={config['username']};"
        f"PWD={config['password']};"
        f"Encrypt={config['encrypt']};"
        f"TrustServerCertificate={config['trust_server_certificate']};"
    )
    try:
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()
        cursor.execute(f"""
            IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = '{table_name}')
            CREATE TABLE {table_name} (
                period VARCHAR(50),
                total_sales FLOAT,
                total_quantity FLOAT,
                calculated_date DATE
            )
        """)
        cursor.execute(f"TRUNCATE TABLE {table_name}")
        conn.commit()
        # Push via SQLAlchemy
        engine = create_engine(f"mssql+pyodbc:///?odbc_connect={conn_str.replace(';', '%3B')}")
        summary_df.to_sql(table_name, engine, if_exists='append', index=False, method='multi')
        conn.close()
        print(f"Pushed {len(summary_df)} rows to `{table_name}`")
    except Exception as e:
        print(f"Push failed for {table_name}: {e}")

# ================================
# MAIN
# ================================
def main(config):
    df, date_col, value_col, quantity_col = load_data(config)
    if df is None: return

    current_date = pd.to_datetime('2025-11-13')

    # Recent (current/previous periods)
    recent_summary = calculate_period_sums(df, date_col, value_col, quantity_col, current_date, is_past=False)
    print("\nRecent Timeframe Sums:")
    print(recent_summary)

    # Past (same periods last year)
    past_summary = calculate_period_sums(df, date_col, value_col, quantity_col, current_date, is_past=True)
    print("\nPast Timeframe Sums (Last Year):")
    print(past_summary)

    # Push to DB
    push_summary_to_db(recent_summary, config, config['recent_summary_table'])
    push_summary_to_db(past_summary, config, config['past_summary_table'])

if __name__ == "__main__":
    main(CONFIG)

Connecting to MSSQL...


C:\Users\Fau\AppData\Local\Temp\ipykernel_25120\3233660669.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(config['sql_query'], conn)


Loaded 27,174 rows from DB
Cleaned data: 27,174 rows

Recent Timeframe Sums:
             period  total_sales  total_quantity calculated_date
0      previous_day          0.0             0.0      2025-11-13
1     previous_week          0.0             0.0      2025-11-13
2    previous_month          0.0             0.0      2025-11-13
3  previous_quarter          0.0             0.0      2025-11-13
4         past_year          0.0             0.0      2025-11-13

Past Timeframe Sums (Last Year):
             period  total_sales  total_quantity calculated_date
0      previous_day          0.0             0.0      2025-11-13
1     previous_week          0.0             0.0      2025-11-13
2    previous_month          0.0             0.0      2025-11-13
3  previous_quarter          0.0             0.0      2025-11-13
4         past_year          0.0             0.0      2025-11-13
Pushed 5 rows to `recent_timeframe_performance_summary`
Pushed 5 rows to `past_timeframe_performance_summary`